<img src="https://raw.githubusercontent.com/Databricks-BR/lab_agosto_2025/refs/heads/main/images/head_lab.png" width="1000px">

##### Descrição e Objetivos

Este projeto tem como objetivo fornecer uma **visualização interativa da inadimplência** no estado de São Paulo, utilizando dados geoespaciais e inteligência artificial para análises avançadas.

##### Controle de Versão do Código

 versão | data | autor | e-mail | alterações |
 --- | --- | --- | --- | --- |
 1.0 | 19-AGO-2025 | Luis Assunção<br>Vinicius Fialho | luis.assuncao@databricks.com<br>vinicius.fialho@databricks.com | Primeira versão  |

##### Descrição do Cluster

Toda a demo pode ser executada utilizando Serverless ou um tipo de instância de sua pereferência.

In [0]:
%pip install --upgrade databricks-sdk==0.36.0 -q
dbutils.library.restartPython()

In [0]:
from databricks.sdk import WorkspaceClient

app_name = "geospatial-viewer"

# Vá na aba do Databricks SQL Warehouse, selecione o SQL Warehouse desejado, copie o ID e cole na variável abaixo
sql_warehouse_id = "<INSIRA-O-SQL-WAREHOUSE-ID-AQUI>"

w = WorkspaceClient()

## Deploy do Aplicativo


Para realizarmos o deploy do aplicativo, utilizaremos o **Databricks Apps**, que são aplicações fáceis de criar e usar dentro da plataforma Databricks, permitindo transformar dados e inteligência artificial em soluções práticas. Elas oferecem uma forma rápida e segura de construir ferramentas interativas para análise, visualização e automação, ajudando equipes a tomar decisões melhores com base nos dados.

Para saber mais sobre Databricks Apps, [acesse a documentação](https://docs.databricks.com/aws/pt/dev-tools/databricks-apps).

Antes de executar a célula a seguir, siga os seguintes passos:

1. Verifique se o **nome completo da tabela Gold** está correto;
2. Altere a variável **genie_space_id** com o ID do seu Genie Space no arquivo lakehouse_app/app.py;
3. Depois de publicar o Dashboard, clique em Share, vá em Embed dashboard, clique em iFrame, copie o link da variável src e cole na variável **src_dashboard** do arquivo lakehouse_app/app.py.

In [0]:
from databricks.sdk.service.apps import AppResource, AppResourceSqlWarehouse, AppResourceSqlWarehouseSqlWarehousePermission, AppResourceServingEndpoint, AppResourceServingEndpointServingEndpointPermission

import os

# Obtém o caminho da pasta de criação do Lakehouse App
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
notebook_folder = os.path.dirname(notebook_path)
path = (f"/Workspace{notebook_folder}/lakehouse_app")

resources = []

# Define os recursos a serem utilizados
sql_resource = AppResource(
  name="sql_warehouse",
  sql_warehouse=AppResourceSqlWarehouse(
    id=sql_warehouse_id,
    permission=AppResourceSqlWarehouseSqlWarehousePermission.CAN_USE
  )
)
resources.append(sql_resource)

# Checa se o Lakehouse App já existe
# Caso o App já exista, apenas será feito o deploy novamente, sem necessidade de criá-lo
if app_name in [app.name for app in w.apps.list()]:
  print(f"App {app_name} já existe")
  app = w.apps.get(app_name)

  app_deploy = w.apps.deploy_and_wait(app_name=app_name, source_code_path=path)
  print(app_deploy.status.message)
  print(app.url)

else:
  print(f"Criando Lakehouse App com o nome {app_name}, esta etapa deve demorar alguns minutos para completar")

  app_created = w.apps.create_and_wait(name=app_name, resources=resources)
  app_deploy = w.apps.deploy_and_wait(app_name=app_name, source_code_path=path)

  print(app_deploy.status.message)
  print(app_created.url)


### Passo importante
Logo após criar o App, navegue na interface do Databricks para a aba Compute > Apps > Selecione o App criado > clique em Authorization > copie o valor do **Service Principal** > vá até o **catálogo de dados** criado na aba Catalog e dê permissão para o Service Principal utilizar os catálogo, schema e tabela em que armazenamos os dados.

Faça o mesmo com o **Genie Space** criado, dê acesso de "Can Run" para o Service Principal.

Após isso, será possível utilizar o aplicativo com todos os dados já integrados.